# 🧬 Project CYBERSPORE - Colab Setup

**Goal**: Run the Metabolic War evolution on Google Colab (faster than local Windows machine).

**What this does**:
1. Clones your OpenVINO fork with Cyberspore code
2. Installs dependencies
3. Compiles the C++ TSSN extension
4. Runs the infection + evolution pipeline

---

## Step 1: Mount Google Drive (Optional)

Mount your Drive to save checkpoints and results.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create a folder for outputs
!mkdir -p /content/drive/MyDrive/Cyberspore

## Step 2: Clone Repository

In [ ]:
!git clone https://github.com/ssdajoker/openvino.git
%cd openvino

## Step 3: Install Dependencies

In [ ]:
# OpenVINO Development Kit (includes C++ headers)
!pip install -q openvino-dev

# Python libraries
!pip install -q numpy pandas tqdm matplotlib seaborn tabulate torch transformers optimum-intel

## Step 4: Build C++ Extension

Compile the CompositeTSSN custom operation.

In [ ]:
import os

# Create build directory
os.makedirs("src/custom_ops/build", exist_ok=True)

# Configure and build
!cd src/custom_ops/build && cmake -DCMAKE_BUILD_TYPE=Release .. && make -j

# Verify
!ls -lh src/custom_ops/build/libopenvino_tssn_extension.so

## Step 5: Download Gemma Model

You need the base Gemma IR. If you already have it locally, upload it to Drive and copy here.

**Option A**: Upload from your machine
`python
from google.colab import files
uploaded = files.upload()  # Upload gemma_ir.zip
!unzip gemma_ir.zip
`

**Option B**: Copy from Drive
`python
!cp -r /content/drive/MyDrive/Cyberspore/gemma_ir .
`

In [ ]:
# TODO: Upload or copy gemma_ir here
# For now, assuming you'll upload manually
print("⚠️ Upload gemma_ir folder before proceeding!")

## Step 6: Run Infection

Inject TSSN nodes into Gemma.

In [ ]:
!python infect_gemma_test.py

## Step 7: Start Evolution

This will run for hours. Monitor the MSE to see progress.

In [ ]:
# Set environment variable to force CPU mode
import os
os.environ['CYBERSPORE_DEVICE'] = 'CPU'

!python evolve_gemma_v4_steady_state.py

## Step 8: Save Results

Copy evolved model to Drive.

In [ ]:
# Copy to Drive
!cp -r gemma_ir_tssn /content/drive/MyDrive/Cyberspore/evolved_models/
!cp evolution_results.json /content/drive/MyDrive/Cyberspore/
!cp evolution_progress.log /content/drive/MyDrive/Cyberspore/

print("✅ Results saved to Google Drive!")

## Step 9: Analyze Results

Load and visualize the evolution progress.

In [ ]:
import json
import matplotlib.pyplot as plt

# Load results
with open('evolution_results.json') as f:
    results = json.load(f)

# Plot MSE over generations
generations = [r['generation'] for r in results]
mse_values = [r['mse'] for r in results]

plt.figure(figsize=(10, 5))
plt.plot(generations, mse_values, linewidth=2)
plt.xlabel('Generation')
plt.ylabel('MSE (Mean Squared Error)')
plt.title('Cyberspore Evolution: MSE vs Generation')
plt.yscale('log')
plt.grid(True, alpha=0.3)
plt.show()

print(f"📈 Final MSE: {mse_values[-1]:.8f}")
print(f"🏆 Best Function: {results[-1]['best_function']}")